In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder, TargetEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, Lars, RANSACRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, make_scorer
import pickle

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [7]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [9]:
df = pd.read_csv(path_train)
X_test = pd.read_csv(path_test)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.55,Ideal,I,SI1,61.7,55.0,1134,5.29,5.31,3.27
1,0.94,Premium,E,SI1,62.4,56.0,4580,6.27,6.24,3.90
2,0.33,Premium,E,SI2,60.0,59.0,594,4.49,4.47,2.69
3,0.31,Ideal,D,VS2,59.4,56.0,879,4.45,4.40,2.63
4,2.01,Very Good,H,SI1,62.8,59.0,17759,7.99,8.04,5.03
...,...,...,...,...,...,...,...,...,...,...
40450,1.50,Very Good,D,SI1,60.7,62.0,11442,7.30,7.33,4.44
40451,1.51,Ideal,D,SI1,61.9,57.0,11834,7.35,7.42,4.57
40452,0.42,Ideal,G,SI2,60.9,56.0,971,4.87,4.89,2.97
40453,2.05,Premium,E,SI2,59.7,59.0,17237,8.39,8.27,4.97


In [10]:
df = df.dropna()
df = df.drop_duplicates()

In [12]:
df.loc[:, 'y'] = np.where(df['y'] >= 15, df['y'] / 10, df['y'])
df.loc[:, 'z'] = np.where(df['z'] >= 15, df['z'] / 10, df['z'])

X_test.loc[:, 'y'] = np.where(X_test['y'] >= 15, X_test['y'] / 10, X_test['y'])
X_test.loc[:, 'z'] = np.where(X_test['z'] >= 15, X_test['z'] / 10, X_test['z'])

In [13]:
X_train = df.drop(['depth', 'table', 'price'], axis = 1)
y_train = np.array(df.price)

In [14]:
# Блок обучения модели

In [15]:
cat_features = ['cut', 'color']
axis_features = ['x','y','z']
num_features = ['carat']

In [16]:
axis_transformer = Pipeline(steps=[
    ('missing_num', KNNImputer(missing_values=0, n_neighbors=3, weights="uniform")),
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', StandardScaler())])

num = Pipeline(steps=[
    ('missing_num', IterativeImputer(missing_values=0, max_iter=20)),
    # ('kbin', KBinsDiscretizer(n_bins=6, strategy='uniform', encode='ordinal')),
    # ('polynom', PolynomialFeatures(3,include_bias=False)),
    ('scaler', StandardScaler())
    ])


clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':8, 'VVS1':7, 'VVS2':6, 'VS1':5, 'VS2':4, 
               'SI1':3, 'SI2':2, 'I1':1}
    }]

clarity_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', MinMaxScaler())
    ])    
    
CT = ColumnTransformer([
        ("axis_transformer", axis_transformer, axis_features),
        ("num", num, num_features), 
        ("cat", OneHotEncoder(sparse_output=False), cat_features),
        ("ordinal_map", clarity_transformer, ['clarity']), 
        ]).set_output(transform='pandas')

display(CT)

ct = CT.fit_transform(X_train)
pd.DataFrame(ct).head().T

ColumnTransformer(transformers=[('axis_transformer',
                                 Pipeline(steps=[('missing_num',
                                                  KNNImputer(missing_values=0,
                                                             n_neighbors=3)),
                                                 ('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['x', 'y', 'z']),
                                ('num',
                                 Pipeline(steps=[('missing_num',
                                                  IterativeImputer(max_iter=20,
                                                                   missing_values=0)),
                                                 ('scaler', StandardScaler())]),
                                 ['carat']),
                                ('cat', OneHotEncoder(sparse_output=False),
                                 ['cut', 'color']),
                                ('ordinal_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'clarity',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': IF      8
VVS1    7
VVS2    6
VS1     5
VS2     4
SI1     3
SI2     2
I1      1
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['clarity'])])

,0,1,2,3,4
axis_transformer__x,-0.392621,0.482265,-1.106813,-1.142523,2.017779
axis_transformer__y,-0.379223,0.457068,-1.134582,-1.197529,2.075696
axis_transformer__z,-0.387434,0.522765,-1.225394,-1.312080,2.155343
axis_transformer__x^2,-0.455152,0.389987,-1.038829,-1.065507,2.219739
axis_transformer__x y,-0.449210,0.377558,-1.050062,-1.086797,2.259234
axis_transformer__x z,-0.453417,0.413345,-1.085823,-1.131204,2.319781
axis_transformer__y^2,-0.443091,0.364538,-1.060774,-1.107458,2.297166
axis_transformer__y z,-0.447467,0.400794,-1.097066,-1.152094,2.360178
axis_transformer__z^2,-0.450426,0.432352,-1.125990,-1.188372,2.404421
num__carat,-0.521333,0.301391,-0.985435,-1.027626,2.558611


In [17]:
model_pipe = Pipeline([
                ('model', TransformedTargetRegressor(
                        regressor = Pipeline(steps=[  
                                ('preproc', CT),
                                ('estimator', SVR(C = 35, gamma = 0.1))
                            ]),
                        transformer=PowerTransformer(method="yeo-johnson"),
                        check_inverse = False)
                )])
model_pipe.fit(X_train, y_train)

Pipeline(steps=[('model',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=Pipeline(steps=[('preproc',
                                                                       ColumnTransformer(transformers=[('axis_transformer',
                                                                                                        Pipeline(steps=[('missing_num',
                                                                                                                         KNNImputer(missing_values=0,
                                                                                                                                    n_neighbors=3)),
                                                                                                                        ('polynom',
                                                                                                                         PolynomialFeatures(include_bias=False)),
                                                                                                                        ('scaler',
                                                                                                                         StandardScaler())]),
                                                                                                        ['x',
                                                                                                         'y',
                                                                                                         'z']),
                                                                                                       ('num',
                                                                                                        Pipeline(st...
                                                                                                        OneHotEncoder(sparse_output=False),
                                                                                                        ['cut',
                                                                                                         'color']),
                                                                                                       ('ordinal_map',
                                                                                                        Pipeline(steps=[('missing_num',
                                                                                                                         SimpleImputer(strategy='most_frequent')),
                                                                                                                        ('ce',
                                                                                                                         OrdinalEncoder(mapping=[{'col': 'clarity',
                                                                                                                                                  'data_type': dtype('O'),
                                                                                                                                                  'mapping': IF      8
VVS1    7
VVS2    6
VS1     5
VS2     4
SI1     3
SI2     2
I1      1
dtype: int64}])),
                                                                                                                        ('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['clarity'])])),
                                                                      ('estimator',
                                                                       SVR(C=35,
                                                                           gamm

In [18]:
# with open('model_pipe.pkl', 'wb') as file:
#     pickle.dump(model_pipe, file)
with open('model_pipe.pkl', 'rb') as file:
    loaded_model_pipe = pickle.load(file) 

In [7]:
# Блок предсказания с использованием тестового набора

In [19]:
y_predict =  loaded_model_pipe.predict(X_test)

In [24]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict

array([8497.55121597, 1581.68600928, 2699.0340199 , ..., 2378.11668094,
       1669.03255264, 1760.0809581 ])